## Imports

In [10]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import numpy as np
#import matplotlib.pyplot as plt

import datetime
import time
import re
import math

## Preprocessing

In [11]:
name = "data/2021-09-29_panda-intention-recognition"


df = pd.read_csv(name+'.csv')

In [3]:
df.head()

,class,z_nose,z_nose_m10,head_posture_x,head_posture_y,z_wrist_l,z_wrist_r,z_wrist_l_m10,z_wrist_r_m10,z_shoulder_l_m10,...,z541,v541,x542,y542,z542,v542,x543,y543,z543,v543
0,handOver,0.168,0.170,0.597,0.490,0.190,0.194,0.190,0.194,0.179,...,-0.030,0.000,0.519,0.409,-0.033,0.000,0.514,0.415,-0.036,0.000
1,handOver,0.161,0.161,0.487,0.383,0.181,0.189,0.181,0.188,0.171,...,-0.043,0.000,0.506,0.384,-0.050,0.000,0.499,0.387,-0.055,0.000
2,handOver,0.158,0.161,0.519,0.379,0.178,0.000,0.178,0.000,0.168,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,handOver,0.155,0.157,0.531,0.431,0.186,0.180,0.185,0.180,0.166,...,-0.026,0.000,0.477,0.387,-0.031,0.000,0.473,0.392,-0.036,0.000
4,handOver,0.153,0.154,0.550,0.423,0.000,0.176,0.000,0.176,0.165,...,-0.024,0.000,0.476,0.385,-0.027,0.000,0.472,0.387,-0.030,0.000


In [4]:
#list(df.columns.values)

In [5]:
# 0 -                                  Class
# 1 - 8                                Berechnungen 
# 9 - 141         # 33 * 4 = 132       Pose            x1 - x33
# 141 - 2013      # 468 * 4 = 1872     Face            x34 - x501
# 2014 - 2097     # 21 * 4 = 84        rightHand       x502 - x522
# 2098 - 2181     # 21 * 4 = 84        leftHand        x523 - x543

# z_nose_mean10                                        Abstand zur person generell
# nose_x_direcction                                    Blickrichtung
# z_wrist_l_mean10                                     Abstand zur Hand
# z_wrist_r_mean10                                     Abstand zur Hand

# Schulter links (x, y)      11                        Damit Stellung des Arms erkannt wird
# Schulter rechts (x, y)     12                         Damit Stellung des Arms erkannt wird
# Ellenbogen links (x, y)    13                        Damit Stellung des Arms erkannt wird
# Ellenbogen rechts (x, y)   14                        Damit Stellung des Arms erkannt wird

# Handgelenk links  (x, y)   15
# Handgelenk rechts (x, y)   16

# 16 * 16 = 256   full
# 9 * 16 = 144    nur eine seite


## Feature Engineering

##### df muss in make_detection gleich sein!
aber ohne "class"

In [6]:
#df = df.loc[:, ["class","z_nose_mean10","nose_x_direction","z_wrist_l_mean10","z_wrist_r_mean10","x11","y11","x12","y12","x13","y13","x14","y14","x15","y15","x16","y16"]]

# Beide Arme ohne Schulter
# df = df.loc[:, ["class","z_nose_mean10","nose_x_direction","z_wrist_l_mean10","z_wrist_r_mean10","x13","y13","x14","y14","x15","y15","x16","y16"]]
# df.head()

# rechte Hand Optimierung
df = df.loc[:, ["class","z_nose_m10","head_posture_y","z_wrist_r_m10", "z_shoulder_l_m10",'z_elbow_r_m10','watchToRobot','shoulder_wrist','handHigh', 'handInnerSide',"x11","y11","x14","y14","x16","y16", 'x502', 'y502', 'x514', 'y514']]
df.head()

,class,z_nose_m10,head_posture_y,z_wrist_r_m10,z_shoulder_l_m10,z_elbow_r_m10,watchToRobot,shoulder_wrist,handHigh,handInnerSide,x11,y11,x14,y14,x16,y16,x502,y502,x514,y514
0,handOver,0.170,0.490,0.194,0.179,0.214,1,0,0,0,0.761,0.478,0.579,0.679,0.532,0.523,0.563,0.422,0.512,0.423
1,handOver,0.161,0.383,0.188,0.171,0.209,1,0,0,0,0.741,0.471,0.546,0.659,0.519,0.501,0.541,0.409,0.503,0.393
2,handOver,0.161,0.379,0.000,0.168,0.205,1,0,0,0,0.736,0.475,0.540,0.659,0.516,0.503,0.000,0.000,0.000,0.000
3,handOver,0.157,0.431,0.180,0.166,0.200,1,0,0,0,0.726,0.475,0.537,0.670,0.506,0.512,0.503,0.382,0.465,0.391
4,handOver,0.154,0.423,0.176,0.165,0.197,1,0,0,1,0.718,0.489,0.530,0.694,0.498,0.522,0.501,0.385,0.463,0.379


make class to numeric values

highFive = 1    
handOver = 2    
negative = 0

In [7]:
row = 0
countHandOver = 0
countHighFive = 0 
countNegative = 0
for val in df["class"]:
    if val == "highFive":
        df.iloc[row, 0] = 1
        countHighFive += 1
    if val == "handOver":
        df.iloc[row, 0] = 2
        countHandOver += 1
    if val == "negative":
        df.iloc[row, 0] = 0
        countNegative += 1
    row +=1

print("HandOver #Rows: " + str(countHandOver))    
print("HighFive #Rows: " + str(countHighFive))
print("Negative #Rows: " + str(countNegative))


HandOver #Rows: 856
HighFive #Rows: 1263
Negative #Rows: 4338


Für jedes Feature rolling 4, 8, 12, 16

#### Pro Feature *16 !

##### no_rolling_features_arr muss in make_detection gleich sein!

In [8]:
no_rolling_features_arr = ['class', 'z_nose_m10', 'head_posture_y', 'watchToRobot', 'x502', 'y502', 'x514', 'y514']

print("Features with Rolling: ")
print(" ")

for col_name in df.columns:
    if col_name not in no_rolling_features_arr:
        #df[col_name + '_moving_avg_12'] = df[col_name].rolling(12).mean()
        #df[col_name + '_moving_sum_12'] = df[col_name].rolling(12).sum()
        #df[col_name + '_moving_var_12'] = df[col_name].rolling(12).var()
        #df[col_name + '_moving_std_12'] = df[col_name].rolling(12).std()

        df[col_name + '_moving_avg_8'] = df[col_name].rolling(8).mean()
        df[col_name + '_moving_sum_8'] = df[col_name].rolling(8).sum()
        df[col_name + '_moving_var_8'] = df[col_name].rolling(8).var()
        df[col_name + '_moving_std_8'] = df[col_name].rolling(8).std()

        #df[col_name + '_moving_avg_16'] = df[col_name].rolling(16).mean()
        #df[col_name + '_moving_sum_16'] = df[col_name].rolling(16).sum()
        #df[col_name + '_moving_var_16'] = df[col_name].rolling(16).var()
        #df[col_name + '_moving_std_16'] = df[col_name].rolling(16).std()

        df[col_name + '_moving_avg_4'] = df[col_name].rolling(4).mean()
        df[col_name + '_moving_sum_4'] = df[col_name].rolling(4).sum()
        df[col_name + '_moving_var_4'] = df[col_name].rolling(4).var()
        df[col_name + '_moving_std_4'] = df[col_name].rolling(4).std()

        print(col_name)

Features with Rolling: 
 
z_wrist_r_m10
z_shoulder_l_m10
z_elbow_r_m10
shoulder_wrist
handHigh
handInnerSide
x11
y11
x14
y14
x16
y16


In [9]:
# Drop rows with NaN values 
df = df.dropna()
#df = df.reset_index()
df.head()

,class,z_nose_m10,head_posture_y,z_wrist_r_m10,z_shoulder_l_m10,z_elbow_r_m10,watchToRobot,shoulder_wrist,handHigh,handInnerSide,...,x16_moving_var_4,x16_moving_std_4,y16_moving_avg_8,y16_moving_sum_8,y16_moving_var_8,y16_moving_std_8,y16_moving_avg_4,y16_moving_sum_4,y16_moving_var_4,y16_moving_std_4
7,2,0.146,0.542,0.161,0.162,0.179,1,0,0,0,...,0.000,0.015,0.543,4.348,0.003,0.051,0.577,2.308,0.003,0.055
8,2,0.144,0.550,0.160,0.162,0.174,1,0,0,0,...,0.000,0.016,0.564,4.512,0.005,0.071,0.618,2.473,0.004,0.062
9,2,0.142,0.585,0.146,0.162,0.165,1,0,0,1,...,0.000,0.015,0.596,4.769,0.009,0.093,0.671,2.685,0.005,0.070
10,2,0.141,0.608,0.000,0.162,0.162,1,0,0,1,...,0.000,0.007,0.631,5.048,0.011,0.105,0.719,2.876,0.004,0.063
11,2,0.140,0.640,0.129,0.161,0.154,1,0,0,1,...,0.000,0.003,0.669,5.352,0.012,0.111,0.761,3.044,0.003,0.055


## Train simple classifier

In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [41]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [43]:
y_test=y_test.astype('int')
y_train=y_train.astype('int')

In [44]:
y_test

4091    0
1756    1
3827    0
3378    0
6285    0
       ..
5216    0
259     2
1546    1
3879    0
1623    1
Name: class, Length: 1935, dtype: int64

In [45]:
df[df['shoulder_wrist']==1] #highFive

,class,z_nose_m10,head_posture_y,z_wrist_r_m10,z_shoulder_l_m10,z_elbow_r_m10,watchToRobot,shoulder_wrist,handHigh,handInnerSide,...,x16_moving_var_4,x16_moving_std_4,y16_moving_avg_8,y16_moving_sum_8,y16_moving_var_8,y16_moving_std_8,y16_moving_avg_4,y16_moving_sum_4,y16_moving_var_4,y16_moving_std_4
939,1,0.130,0.650,0.100,0.148,0.121,1,1,1,1,...,0.000,0.001,0.786,6.286,0.000,0.003,0.788,3.152,0.000,0.002
940,1,0.129,0.644,0.103,0.148,0.117,1,1,1,1,...,0.000,0.004,0.786,6.290,0.000,0.003,0.788,3.153,0.000,0.002
941,1,0.130,0.652,0.104,0.148,0.116,1,1,1,1,...,0.000,0.005,0.787,6.294,0.000,0.002,0.788,3.152,0.000,0.002
942,1,0.130,0.658,0.106,0.149,0.116,1,1,1,1,...,0.000,0.004,0.787,6.294,0.000,0.002,0.787,3.148,0.000,0.003
943,1,0.129,0.648,0.106,0.150,0.116,1,1,1,1,...,0.000,0.003,0.787,6.293,0.000,0.002,0.785,3.141,0.000,0.002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6084,0,0.139,0.504,0.160,0.142,0.176,1,1,1,0,...,0.000,0.001,0.723,5.780,0.000,0.003,0.724,2.897,0.000,0.004
6085,0,0.139,0.506,0.160,0.142,0.177,1,1,1,0,...,0.000,0.001,0.724,5.791,0.000,0.004,0.726,2.905,0.000,0.004
6086,0,0.139,0.506,0.159,0.142,0.177,1,1,1,0,...,0.000,0.001,0.725,5.797,0.000,0.004,0.727,2.910,0.000,0.003
6087,0,0.139,0.490,0.159,0.142,0.177,1,1,1,0,...,0.000,0.000,0.725,5.802,0.000,0.004,0.728,2.914,0.000,0.002


In [46]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [47]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000)),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=10)),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
} 

In [48]:
start = time.time()

fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

end = time.time()
print(f"Runtime of the program is {end - start}")

Runtime of the program is 29.29164743423462


In [49]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression(max_iter=1000))]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [63]:
rfc = RandomForestClassifier(max_depth=10)
rfc.fit(X_train, y_train)

RandomForestClassifier(max_depth=10)

In [58]:
#[estimator.tree_.max_depth for estimator in model.estimators_]

In [142]:
fit_models['rc'].predict(X_test)

array([0, 2, 1, ..., 1, 2, 2])

## Evaluate and Serialize Model 

In [61]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score # Accuracy metrics 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pickle 

In [144]:
print("        accuracy           precision            recall              f1")
for algo, model in fit_models.items():
    y_pred = model.predict(X_test)
    print(algo, accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='micro'), recall_score(y_test, y_pred, average='micro'), f1_score(y_test, y_pred, average='micro'))


        accuracy           precision            recall              f1
lr 0.9527950310559006 0.9527950310559006 0.9527950310559006 0.9527950310559006
rc 0.9416149068322981 0.9416149068322981 0.9416149068322981 0.9416149068322981
rf 0.991304347826087 0.991304347826087 0.991304347826087 0.991304347826087
gb 0.9919254658385093 0.9919254658385093 0.9919254658385093 0.9919254658385093


In [67]:
y_pred = rfc.predict(X_test)
print(accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='micro'), recall_score(y_test, y_pred, average='micro'), f1_score(y_test, y_pred, average='micro'))

y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cmWithCosts = cm*costs
print(algo)
print(cm , end =" ")
print("costs: " + str(sum(sum(cmWithCosts))))
print(" ")

0.979328165374677 0.979328165374677 0.979328165374677 0.979328165374677
gb
[[1289    3    0]
 [  18  377    0]
 [   6    4  238]] costs: 40
 


##### Eigene Kosten dazu berechnen

In [66]:
costs = [[0, 4, 4], 
        [1, 0, 4],
        [1, 1, 0]]

for algo, model in fit_models.items():
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    cmWithCosts = cm*costs
    print(algo)
    print(cm , end =" ")
    print("costs: " + str(sum(sum(cmWithCosts))))
    print(" ")
    

lr
[[1259   18   15]
 [  40  346    9]
 [  21    4  223]] costs: 233
 
rc
[[1252   19   21]
 [  72  318    5]
 [  35    5  208]] costs: 292
 
rf
[[1290    2    0]
 [  15  379    1]
 [   7    1  240]] costs: 35
 
gb
[[1289    3    0]
 [  18  377    0]
 [   6    4  238]] costs: 40
 


In [146]:
print("0: negative")
print("1: highFive")
print("2: handOver")
print(" ")

for algo, model in fit_models.items():
    y_pred = model.predict(X_test)
    print(algo, end= " ")
    print(classification_report(y_test, y_pred))

0: negative
1: highFive
2: handOver
 
lr               precision    recall  f1-score   support

           0       0.96      0.97      0.96       912
           1       0.95      0.90      0.93       374
           2       0.94      0.96      0.95       324

    accuracy                           0.95      1610
   macro avg       0.95      0.94      0.95      1610
weighted avg       0.95      0.95      0.95      1610

rc               precision    recall  f1-score   support

           0       0.94      0.97      0.96       912
           1       0.97      0.87      0.92       374
           2       0.92      0.94      0.93       324

    accuracy                           0.94      1610
   macro avg       0.94      0.93      0.93      1610
weighted avg       0.94      0.94      0.94      1610

rf               precision    recall  f1-score   support

           0       0.99      1.00      0.99       912
           1       1.00      0.97      0.99       374
           2       0.99     

In [147]:
with open('gradientBoost_no-rolling.pkl', 'wb') as f:
    pickle.dump(fit_models['lr'], f)

## Get Importances of Features from LogisticRegression

In [148]:
listWithFeatureNames = []
for col in df.columns:
    listWithFeatureNames.append(col)


In [149]:
class_weight={0:0.5,1:1, 2:1}

In [153]:
# define the model
model = LogisticRegression(max_iter=10000, multi_class="multinomial", class_weight=class_weight)
# fit the model
model.fit(X_train, y_train)
# get importance
importance = model.coef_[0]

limit = 0.5
final_feature_list = []

# summarize feature importance
for i,v in enumerate(importance):
    #print('Feature: %0d, Score: %.5f' % (i,v))
    if v >= limit:
        final_feature_list.append(i)
        featureName = listWithFeatureNames[1]
        print('Feature: ' + listWithFeatureNames[i] + ', Score: %.5f' % (v))
        

Feature: class, Score: 0.60905
Feature: head_posture_y, Score: 2.14986
Feature: z_shoulder_l_m10, Score: 0.51321
Feature: z_wrist_r_m10_moving_var_8, Score: 0.71718
Feature: z_wrist_r_m10_moving_avg_4, Score: 1.24413
Feature: z_shoulder_l_m10_moving_avg_8, Score: 3.34338
Feature: z_shoulder_l_m10_moving_var_8, Score: 0.79841
Feature: z_shoulder_l_m10_moving_avg_4, Score: 1.66175
Feature: z_elbow_r_m10_moving_avg_8, Score: 1.28365
Feature: z_elbow_r_m10_moving_var_8, Score: 0.61456
Feature: z_elbow_r_m10_moving_avg_4, Score: 0.63143
Feature: shoulder_wrist_moving_avg_8, Score: 0.64365
Feature: x11_moving_avg_8, Score: 1.76196
Feature: y11_moving_var_8, Score: 1.22279
Feature: y11_moving_avg_4, Score: 0.99723
Feature: y11_moving_var_4, Score: 1.31397
Feature: y14_moving_var_8, Score: 1.50089
Feature: y14_moving_var_4, Score: 1.44132
Feature: x16_moving_avg_8, Score: 1.31611
Feature: x16_moving_var_8, Score: 1.03638
Feature: x16_moving_var_4, Score: 0.62822
Feature: y16_moving_avg_8, Scor

In [151]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.92      0.93       912
           1       0.92      0.92      0.92       374
           2       0.85      0.92      0.88       324

    accuracy                           0.92      1610
   macro avg       0.90      0.92      0.91      1610
weighted avg       0.92      0.92      0.92      1610



# Ab hier Entwicklung

### Train Decision Tree Classifier with normal Splits

In [278]:
#decisionTree Classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

seed = 123

tree_model = DecisionTreeClassifier(criterion='entropy',random_state=seed)
tree_model.fit(X_train,y_train)
y_pred = tree_model.predict(X_test)

In [279]:
cm = confusion_matrix(y_test,y_pred)
score = (cm[0][0] + cm[1][1] + cm[2][2]) / sum(sum(cm))
print('Accuracy: ',score)

Accuracy:  0.9460093896713615


In [280]:
y_pred = tree_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96      1213
           1       0.94      0.91      0.92       435
           2       0.91      0.93      0.92       482

    accuracy                           0.95      2130
   macro avg       0.94      0.93      0.94      2130
weighted avg       0.95      0.95      0.95      2130



In [69]:
with open('gradientBoost_no-rolling.pkl', 'wb') as f:
    pickle.dump(tree_model, f)

### Train Decision Tree Classifier with leave one out split

In [70]:
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold

In [73]:
y=y.astype('int')

# Leave one out
X_arr = np.array(X)
y_arr = np.array(y)

X_train_loo = []
X_test_loo = []
y_train_loo = []
y_test_loo = [] 

loo = LeaveOneOut()
loo.get_n_splits(X_arr)

tree_model_loo = DecisionTreeClassifier(criterion='entropy',random_state=seed)
results = []
i = 0
for train_index, test_index in loo.split(X_arr):
    if(i == 999):
        break
        
    X_train_loo = X_arr[train_index]
    X_test_loo = X_arr[test_index]
    y_train_loo = y_arr[train_index]
    y_test_loo = y_arr[test_index]
    
    tree_model_loo.fit(X_train_loo,y_train_loo)
    y_pred_loo = tree_model_loo.predict(X_test_loo)
    
    if(y_pred_loo == y_test_loo):
        results.append(1)
    else:
        results.append(0)
    i += 1
print('Accuracy: ' + str(sum(results)/len(results)))

Accuracy: 0.9519519519519519


In [74]:
y_pred = tree_model_loo.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1051
           1       1.00      1.00      1.00       412
           2       1.00      1.00      1.00       326

    accuracy                           1.00      1789
   macro avg       1.00      1.00      1.00      1789
weighted avg       1.00      1.00      1.00      1789



In [75]:
# get importance
importance = tree_model_loo.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
#pyplot.bar([x for x in range(len(importance))], importance)
#pyplot.show()

Feature: 0, Score: 0.00327
Feature: 1, Score: 0.00510
Feature: 2, Score: 0.02828
Feature: 3, Score: 0.00579
Feature: 4, Score: 0.00204
Feature: 5, Score: 0.00834
Feature: 6, Score: 0.01087
Feature: 7, Score: 0.00057
Feature: 8, Score: 0.00000
Feature: 9, Score: 0.00629
Feature: 10, Score: 0.01693
Feature: 11, Score: 0.00848
Feature: 12, Score: 0.00000
Feature: 13, Score: 0.00760
Feature: 14, Score: 0.00000
Feature: 15, Score: 0.00088
Feature: 16, Score: 0.00000
Feature: 17, Score: 0.00088
Feature: 18, Score: 0.00000
Feature: 19, Score: 0.00153
Feature: 20, Score: 0.00000
Feature: 21, Score: 0.00771
Feature: 22, Score: 0.01781
Feature: 23, Score: 0.00000
Feature: 24, Score: 0.00000
Feature: 25, Score: 0.05162
Feature: 26, Score: 0.00162
Feature: 27, Score: 0.02210
Feature: 28, Score: 0.00425
Feature: 29, Score: 0.00397
Feature: 30, Score: 0.01818
Feature: 31, Score: 0.00000
Feature: 32, Score: 0.00034
Feature: 33, Score: 0.10470
Feature: 34, Score: 0.00000
Feature: 35, Score: 0.05951
Fe

In [109]:
newDict = {}
for i, newVal in enumerate(importance):
    if len(newDict) < 10:
        #maxList.append(newVal)
        newDict[i] = newVal
        continue
    if len(newDict) >= 10:
        minElement = min(newDict.values())
        key_min = min(newDict.keys(), key=(lambda k: newDict[k]))
        print(key_min)
        if newVal > minElement:
            
            newDict.pop(key_min)
            newDict[i] = newVal

8
7
4
4
0
0
0
0
0
0
0
0
1
3
3
3
9
9
13
13
13
21
21
21
5
5
11
11
11
11
11
11
11
6
6
6
6
6
6
6
10
10
22
22
22
22
22
22
22
22
22
22
22
22
22
22
30
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
27
27
27
27
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49


In [110]:
newDict

{2: 0.02828238606166862,
 25: 0.051621752589082574,
 33: 0.1047008680936074,
 35: 0.059509319794796296,
 42: 0.15424589048194387,
 49: 0.02390176961573761,
 51: 0.03778525539630504,
 66: 0.03611808571629437,
 122: 0.027379933462803124,
 126: 0.0917002981075556}

## Feature Elimination 

In [206]:
import numpy as np 
from sklearn.ensemble import RandomForestClassifier

In [207]:
# Get Feature Importance
rf_classifier = RandomForestClassifier(n_estimators=1000, max_depth=2,
                             random_state=0)
rf_classifier.fit(X, y)  

RandomForestClassifier(max_depth=2, n_estimators=1000, random_state=0)

In [208]:
features = X.columns
importances = rf_classifier.feature_importances_.shape
ft_importances = pd.DataFrame({'Features': X.columns, 'Importances': rf_classifier.feature_importances_}).sort_values('Importances',ascending=False)

ft_importances

,Features,Importances
33,z_wrist_r_mean10_moving_avg_12,0.074
34,z_wrist_r_mean10_moving_sum_12,0.073
2,z_wrist_r_mean10,0.044
42,z_shoulder_l_m10_moving_sum_12,0.043
41,z_shoulder_l_m10_moving_avg_12,0.042
...,...,...
83,v12_moving_var_12,0.000
80,z12_moving_std_8,0.000
127,x16_moving_var_8,0.000
128,x16_moving_std_8,0.000


In [209]:
print(ft_importances.to_string())

                           Features  Importances
33   z_wrist_r_mean10_moving_avg_12        0.074
34   z_wrist_r_mean10_moving_sum_12        0.073
2                  z_wrist_r_mean10        0.044
42   z_shoulder_l_m10_moving_sum_12        0.043
41   z_shoulder_l_m10_moving_avg_12        0.042
37    z_wrist_r_mean10_moving_avg_8        0.040
45    z_shoulder_l_m10_moving_avg_8        0.040
38    z_wrist_r_mean10_moving_sum_8        0.040
46    z_shoulder_l_m10_moving_sum_8        0.036
66                y12_moving_sum_12        0.030
49   z_shoulder_r_m10_moving_avg_12        0.028
65                y12_moving_avg_12        0.027
26   nose_x_direction_moving_sum_12        0.023
3                  z_shoulder_l_m10        0.023
69                 y12_moving_avg_8        0.021
25   nose_x_direction_moving_avg_12        0.021
50   z_shoulder_r_m10_moving_sum_12        0.021
53    z_shoulder_r_m10_moving_avg_8        0.020
70                 y12_moving_sum_8        0.020
4                  z

In [210]:
test_size = 0.33
seed = 123

In [212]:
from sklearn.tree import DecisionTreeClassifier

# Recursive Feature Elimination
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
best_features = np.array(ft_importances.index)

limit = 0.01

features = np.array(features)
feature_list = []
last_score = 0
for feature in best_features:
    feature_list.append(feature)
    
    # Train Model
    tree_classifier = DecisionTreeClassifier(criterion='entropy',random_state=seed, max_depth=7)
    tree_classifier.fit(X_train.iloc[:,feature_list],y_train)
    y_pred = tree_classifier.predict(X_test.iloc[:,feature_list])
    
    # Evaluation
    cm = confusion_matrix(y_test,y_pred)
    score = (cm[0][0] + cm[1][1] + cm[2][2]) / sum(sum(cm))
    diff = score - last_score
    
    
    print('Feature List: ',list(features[feature_list]))
    print('  Score: ' + str(score))
    print('  Difference Last Score: ' + str(diff))
    
    if(diff > limit):
        print('  Keep Feature: ',features[feature])
        last_score = score
    elif(diff == limit):
        print('  Delete Feature: ',features[feature])
        del feature_list[-1]
    else:
        print('  Delete Feature: ',features[feature])
        del feature_list[-1]
    print('\n')

Feature List:  ['z_wrist_r_mean10_moving_avg_12']
  Score: 0.6341463414634146
  Difference Last Score: 0.6341463414634146
  Keep Feature:  z_wrist_r_mean10_moving_avg_12


Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10_moving_sum_12']
  Score: 0.6341463414634146
  Difference Last Score: 0.0
  Delete Feature:  z_wrist_r_mean10_moving_sum_12


Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10']
  Score: 0.717479674796748
  Difference Last Score: 0.08333333333333337
  Keep Feature:  z_wrist_r_mean10


Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10', 'z_shoulder_l_m10_moving_sum_12']
  Score: 0.7865853658536586
  Difference Last Score: 0.06910569105691056
  Keep Feature:  z_shoulder_l_m10_moving_sum_12


Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10', 'z_shoulder_l_m10_moving_sum_12', 'z_shoulder_l_m10_moving_avg_12']
  Score: 0.7860772357723578
  Difference Last Score: -0.000508130081300795
  Delete Feature:

Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10', 'z_shoulder_l_m10_moving_sum_12', 'nose_x_direction_moving_sum_12', 'z_shoulder_r_m10_moving_sum_12', 'x16_moving_avg_12']
  Score: 0.8470528455284553
  Difference Last Score: -0.002032520325203291
  Delete Feature:  x16_moving_avg_12


Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10', 'z_shoulder_l_m10_moving_sum_12', 'nose_x_direction_moving_sum_12', 'z_shoulder_r_m10_moving_sum_12', 'z_wrist_r_mean10_moving_var_12']
  Score: 0.8419715447154471
  Difference Last Score: -0.0071138211382114624
  Delete Feature:  z_wrist_r_mean10_moving_var_12


Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10', 'z_shoulder_l_m10_moving_sum_12', 'nose_x_direction_moving_sum_12', 'z_shoulder_r_m10_moving_sum_12', 'x16']
  Score: 0.8633130081300813
  Difference Last Score: 0.014227642276422703
  Keep Feature:  x16


Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10', 'z_shoulder_l

Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10', 'z_shoulder_l_m10_moving_sum_12', 'nose_x_direction_moving_sum_12', 'z_shoulder_r_m10_moving_sum_12', 'x16', 'x14_moving_sum_12', 'z_wrist_r_mean10_moving_std_8', 'x12_moving_sum_12']
  Score: 0.8744918699186992
  Difference Last Score: -0.016260162601625994
  Delete Feature:  x12_moving_sum_12


Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10', 'z_shoulder_l_m10_moving_sum_12', 'nose_x_direction_moving_sum_12', 'z_shoulder_r_m10_moving_sum_12', 'x16', 'x14_moving_sum_12', 'z_wrist_r_mean10_moving_std_8', 'z_nose_mean10_moving_avg_8']
  Score: 0.8927845528455285
  Difference Last Score: 0.002032520325203291
  Delete Feature:  z_nose_mean10_moving_avg_8


Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10', 'z_shoulder_l_m10_moving_sum_12', 'nose_x_direction_moving_sum_12', 'z_shoulder_r_m10_moving_sum_12', 'x16', 'x14_moving_sum_12', 'z_wrist_r_mean10_moving_std_8', 'z_nose_mean10

Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10', 'z_shoulder_l_m10_moving_sum_12', 'nose_x_direction_moving_sum_12', 'z_shoulder_r_m10_moving_sum_12', 'x16', 'x14_moving_sum_12', 'z_wrist_r_mean10_moving_std_8', 'v14_moving_std_12']
  Score: 0.8943089430894309
  Difference Last Score: 0.0035569105691056757
  Delete Feature:  v14_moving_std_12


Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10', 'z_shoulder_l_m10_moving_sum_12', 'nose_x_direction_moving_sum_12', 'z_shoulder_r_m10_moving_sum_12', 'x16', 'x14_moving_sum_12', 'z_wrist_r_mean10_moving_std_8', 'y16_moving_var_8']
  Score: 0.866869918699187
  Difference Last Score: -0.02388211382113825
  Delete Feature:  y16_moving_var_8


Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10', 'z_shoulder_l_m10_moving_sum_12', 'nose_x_direction_moving_sum_12', 'z_shoulder_r_m10_moving_sum_12', 'x16', 'x14_moving_sum_12', 'z_wrist_r_mean10_moving_std_8', 'z16_moving_avg_12']
  Score: 0.882

Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10', 'z_shoulder_l_m10_moving_sum_12', 'nose_x_direction_moving_sum_12', 'z_shoulder_r_m10_moving_sum_12', 'x16', 'x14_moving_sum_12', 'z_wrist_r_mean10_moving_std_8', 'z14_moving_sum_12']
  Score: 0.8846544715447154
  Difference Last Score: -0.0060975609756097615
  Delete Feature:  z14_moving_sum_12


Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10', 'z_shoulder_l_m10_moving_sum_12', 'nose_x_direction_moving_sum_12', 'z_shoulder_r_m10_moving_sum_12', 'x16', 'x14_moving_sum_12', 'z_wrist_r_mean10_moving_std_8', 'x14_moving_std_12']
  Score: 0.8724593495934959
  Difference Last Score: -0.018292682926829285
  Delete Feature:  x14_moving_std_12


Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10', 'z_shoulder_l_m10_moving_sum_12', 'nose_x_direction_moving_sum_12', 'z_shoulder_r_m10_moving_sum_12', 'x16', 'x14_moving_sum_12', 'z_wrist_r_mean10_moving_std_8', 'x14_moving_std_8']
  Score: 0

Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10', 'z_shoulder_l_m10_moving_sum_12', 'nose_x_direction_moving_sum_12', 'z_shoulder_r_m10_moving_sum_12', 'x16', 'x14_moving_sum_12', 'z_wrist_r_mean10_moving_std_8', 'v16_moving_var_12']
  Score: 0.8887195121951219
  Difference Last Score: -0.002032520325203291
  Delete Feature:  v16_moving_var_12


Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10', 'z_shoulder_l_m10_moving_sum_12', 'nose_x_direction_moving_sum_12', 'z_shoulder_r_m10_moving_sum_12', 'x16', 'x14_moving_sum_12', 'z_wrist_r_mean10_moving_std_8', 'v16_moving_std_12']
  Score: 0.8887195121951219
  Difference Last Score: -0.002032520325203291
  Delete Feature:  v16_moving_std_12


Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10', 'z_shoulder_l_m10_moving_sum_12', 'nose_x_direction_moving_sum_12', 'z_shoulder_r_m10_moving_sum_12', 'x16', 'x14_moving_sum_12', 'z_wrist_r_mean10_moving_std_8', 'v12']
  Score: 0.8922764227642

  Delete Feature:  x16_moving_std_8


Feature List:  ['z_wrist_r_mean10_moving_avg_12', 'z_wrist_r_mean10', 'z_shoulder_l_m10_moving_sum_12', 'nose_x_direction_moving_sum_12', 'z_shoulder_r_m10_moving_sum_12', 'x16', 'x14_moving_sum_12', 'z_wrist_r_mean10_moving_std_8', 'v16_moving_std_8']
  Score: 0.8907520325203252
  Difference Last Score: 0.0
  Delete Feature:  v16_moving_std_8




In [216]:
final_features = feature_list
print(final_features)

[33, 2, 42, 26, 50, 13, 90, 40]
[33, 2, 42, 26, 50, 13, 90, 40]


In [ ]:
with open(name+'.pkl', 'wb') as f:
    pickle.dump(fit_models['gb'], f)